In [ ]:
%tensorflow_version 1.x
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import gym
import os


BASE_PATH = './'
env = gym.make("Boxing-v0")

In [ ]:
def preprocess_observation(observation):
    img = observation[1:192:2, ::2]
    img = img.mean(axis=2)
    img = (img // 3 - 128).astype(np.int8)
    return img.reshape(96, 80, 1)

In [ ]:
input_height = 96
input_width = 80
input_channels = 1

# filter count number
conv_n_maps = [32, 64, 64]

# size of convolution filter
conv_kernel_sizes = [(8,8), (4,4), (3,3)] 

# strides
conv_strides = [4, 2, 1]

# padding
conv_paddings = ["SAME", "SAME", "SAME"]

conv_activation = [tf.nn.relu] * 3

# input dimension of conv3 has 64 maps of 12x10 each
n_hidden_in = 64 * 12 * 10  

# size of fully connected 1
n_hidden = 512
hidden_activation = tf.nn.relu

# 18 discrete actions are available
n_outputs = env.action_space.n  

# Lazy initializing
initializer = tf.variance_scaling_initializer()

In [ ]:
def q_network(X_state, name):
    prev_layer = X_state / 128.0
    with tf.variable_scope(name) as scope:
        dqn_parameters = zip(conv_n_maps, conv_kernel_sizes, conv_strides, conv_paddings, conv_activation)
        for n_maps, kernel_size, strides, padding, activation in dqn_parameters:
            prev_layer = tf.layers.conv2d(
                prev_layer, filters=n_maps, kernel_size=kernel_size,
                strides=strides, padding=padding, activation=activation,
                kernel_initializer=initializer)
        last_conv_layer_flat = tf.reshape(prev_layer, shape=[-1, n_hidden_in])
        hidden = tf.layers.dense(last_conv_layer_flat, n_hidden,
                                 activation=hidden_activation,
                                 kernel_initializer=initializer)
        outputs = tf.layers.dense(hidden, n_outputs,
                                  kernel_initializer=initializer)
    trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                       scope=scope.name)
    trainable_vars_by_name = {var.name[len(scope.name):]: var
                              for var in trainable_vars}
    return outputs, trainable_vars_by_name

In [5]:
# Input Placeholder
X_state = tf.placeholder(tf.float32, shape=[None, input_height, input_width,
                                            input_channels])

online_q_values, online_vars = q_network(X_state, name="q_networks/online")
target_q_values, target_vars = q_network(X_state, name="q_networks/target")

copy_ops = [target_var.assign(online_vars[var_name])
            for var_name, target_var in target_vars.items()]
copy_online_to_target = tf.group(*copy_ops)

learning_rate = 0.001
momentum = 0.95


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [6]:
with tf.variable_scope("train"):
    X_action = tf.placeholder(tf.int32, shape=[None])
    y = tf.placeholder(tf.float32, shape=[None, 1])
    q_value = tf.reduce_sum(online_q_values * tf.one_hot(X_action, n_outputs),
                            axis=1, keepdims=True)
    error = tf.abs(y - q_value)

    clipped_error = tf.clip_by_value(error, 0.0, 1.0)
    linear_error = 2 * (error - clipped_error)
    loss = tf.reduce_mean(tf.square(clipped_error) + linear_error)

    global_step = tf.Variable(0, trainable=False, name='global_step')
    
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True)
    training_op = optimizer.minimize(loss, global_step=global_step)


# Run all initializers
init = tf.global_variables_initializer()

# Checkpoint Saver
saver = tf.train.Saver()



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
class ReplayMemory:
    def __init__(self, maxlen):
        self.maxlen = maxlen
        self.buf = np.empty(shape=maxlen, dtype=np.object)
        self.index = 0
        self.length = 0

    def append(self, data):
        self.buf[self.index] = data
        self.length = min(self.length + 1, self.maxlen)
        self.index = (self.index + 1) % self.maxlen

    def sample(self, batch_size, with_replacement=True):
        if with_replacement:
            indices = np.random.randint(self.length, size=batch_size)  # faster
        else:
            indices = np.random.permutation(self.length)[:batch_size]
        return self.buf[indices]


In [ ]:
replay_memory_size = 500000
replay_memory = ReplayMemory(replay_memory_size)

In [ ]:
def sample_memories(batch_size):
    cols = [[], [], [], [], []]  # 5-tuple state, action, reward, next_state, continue
    for memory in replay_memory.sample(batch_size):
        for col, value in zip(cols, memory):
            col.append(value)
    cols = [np.array(col) for col in cols]
    return cols[0], cols[1], cols[2].reshape(-1, 1), cols[3], cols[4].reshape(-1, 1)


In [ ]:
eps_min = 0.1
eps_max = 1.0
eps_decay_steps = 2000000

def epsilon_greedy(q_values, step):
    epsilon = max(eps_min, eps_max - (eps_max - eps_min) * step / eps_decay_steps)
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:
        return np.argmax(q_values)  # optimal action


In [ ]:
n_steps = 5000000  # total number of training steps
training_start = 10000  # start training after 10,000 game iterations
training_interval = 4  # run a training step every 4 game iterations
save_steps = 1000  # save the model every 1,000 training steps
copy_steps = 10000  # copy online DQN to target DQN every 10,000 training steps
discount_rate = 0.99
skip_start = 0  # Skip the start of every game (it's just waiting time).
batch_size = 50 # Number of training instances
iteration = 0  # game iterations
checkpoint_path = f"{BASE_PATH}dqn.ckpt"
done = True  # env needs to be reset

loss_val = np.infty
game_length = 0
total_max_q = 0
mean_max_q = 0.0


In [ ]:
def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,


def plot_animation(frames, repeat=False, interval=40):
    plt.close()  # or else nbagg sometimes plots in the previous cell
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    return animation.FuncAnimation(fig, update_scene, fargs=(frames, patch), frames=len(frames), repeat=repeat, interval=interval)


def save_animation(animation, filename):
    animation.save(filename)


In [ ]:
saved = set()
def play_and_record(percent):
    frames = []
    n_max_steps = 100000

    obs = env.reset()
    for step in range(n_max_steps):
        state = preprocess_observation(obs)

        # Online DQN evaluates what to do
        q_values = online_q_values.eval(feed_dict={X_state: [state]})
        action = np.argmax(q_values)

        # Online DQN plays
        obs, reward, done, info = env.step(action)

        img = env.render(mode="rgb_array")
        frames.append(img)

        if done:
            video = plot_animation(frames)
            name = f"{BASE_PATH}boxing_record_{percent}.mp4"
            save_animation(video, name)
            break

record_first_time = False

In [ ]:
with tf.Session() as sess:
    if os.path.isfile(f"{checkpoint_path}.index"):
        saver.restore(sess, checkpoint_path)
    else:
        init.run()
        copy_online_to_target.run()
    while True:
        step = global_step.eval()
        if step >= n_steps:
            break
        iteration += 1
        print("\rIteration {}\tTraining step {}/{} ({:.1f})%\tLoss {:5f}\tMean Max-Q {:5f}   ".format(
            iteration, step, n_steps, step * 100 / n_steps, loss_val, mean_max_q), end="")
        if done:  # game over, start again
            obs = env.reset()
            for skip in range(skip_start):  # skip the start of each game
                obs, reward, done, info = env.step(0)
            state = preprocess_observation(obs)

        total_perc = int(step * 100 / n_steps)
        if record_first_time == True or (total_perc % 10 == 0 and total_perc not in saved):
            play_and_record(total_perc)
            saved.add(total_perc)
            record_first_time = False

        # Online DQN evaluates what to do
        q_values = online_q_values.eval(feed_dict={X_state: [state]})
        action = epsilon_greedy(q_values, step)

        # Online DQN plays
        obs, reward, done, info = env.step(action)
        next_state = preprocess_observation(obs)

        # Let's memorize what happened
        replay_memory.append((state, action, reward, next_state, 1.0 - done))
        state = next_state

        # Compute statistics for tracking progress
        total_max_q += q_values.max()
        game_length += 1
        if done:
            mean_max_q = total_max_q / game_length
            total_max_q = 0.0
            game_length = 0

        if iteration < training_start or iteration % training_interval != 0:
            continue  # only train after warmup period and at regular intervals

        # Sample memories and use the target DQN to produce the target Q-Value
        X_state_val, X_action_val, rewards, X_next_state_val, continues = (
            sample_memories(batch_size))
        next_q_values = target_q_values.eval(feed_dict={X_state: X_next_state_val})
        
        max_next_q_values = np.max(next_q_values, axis=1, keepdims=True)

        # Q Value
        y_val = rewards + continues * discount_rate * max_next_q_values
        
        # Train the online DQN
        _, loss_val = sess.run([training_op, loss], feed_dict={
            X_state: X_state_val, X_action: X_action_val, y: y_val})

        # Regularly copy the online DQN to the target DQN
        if step % copy_steps == 0:
            copy_online_to_target.run()

        # And save regularly
        if step % save_steps == 0:
            saver.save(sess, checkpoint_path)

Iteration 5434	Training step 0/5000000 (0.0)%	Loss   inf	Mean Max-Q 0.271933   

In [ ]:
with tf.Session() as sess:
    if os.path.isfile(f"{checkpoint_path}.index"):
        saver.restore(sess, checkpoint_path)
    else:
        init.run()
        copy_online_to_target.run()
    play_and_record(100)